In [1]:
import re
import pandas as pd

In [2]:
f = open('chat.txt','r', encoding='utf-8')

In [3]:
data = f.read()

In [4]:
data

'User: Hi, can you tell me about Python?,\nAI: Sure! Python is a popular programming language known for its readability,\nUser: What can I use it for?,\nAI: You can use Python for web development, data analysis, AI, and more,'

In [18]:
lines = data.split('\n')
df = pd.DataFrame({'user_message': lines})

In [19]:
df.head()

,user_message
0,"User: Hi, can you tell me about Python?,"
1,AI: Sure! Python is a popular programming lang...
2,"User: What can I use it for?,"
3,"AI: You can use Python for web development, da..."


In [20]:
df.shape

(4, 1)

In [21]:
# Split user and message
users = []
messages = []

for message in df['user_message']:
    match = re.match(r'^(\w+):\s(.+)', message.strip())
    if match:
        users.append(match.group(1))       
        messages.append(match.group(2))    
df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)

df.head()

,user,message
0,User,"Hi, can you tell me about Python?,"
1,AI,Sure! Python is a popular programming language...
2,User,"What can I use it for?,"
3,AI,"You can use Python for web development, data a..."
